# Elasticsearch 日本語用インデックスの登録と分析
3.2節で説明したElasticsearch サーバーの起動が前提です。

In [ ]:
# Elasticsearchインスタンスの生成

from elasticsearch import Elasticsearch
es = Elasticsearch()

In [ ]:
# リスト3.3.3 日本語用インデックスの登録
# インデックス作成用JSONの定義
create_index = {
    "settings": {
        "analysis": {
            "filter": {
                "synonyms_filter": { # 同義語フィルターの定義
                    "type": "synonym",
                    "synonyms": [ #同義語リストの定義 (今は空の状態)
                        ]
                }
            },
            "tokenizer": {
                "kuromoji_w_dic": { # カスタム形態素解析の定義
                "type": "kuromoji_tokenizer", # kromoji_tokenizerをベースにする
                    # ユーザー辞書としてmy_jisho.dicを追加  
                    "user_dictionary": "my_jisho.dic" 
                }
            },
            "analyzer": {
                "jpn-search": { # 検索用アナライザの定義
                    "type": "custom",
                    "char_filter": [
                        "icu_normalizer", # 文字単位の正規化
                        "kuromoji_iteration_mark" # 繰り返し文字の正規化
                    ],
                    "tokenizer": "kuromoji_w_dic", # 辞書付きkuromoji形態素解析
                    "filter": [
                        "synonyms_filter", # 同義語展開
                        "kuromoji_baseform", # 活用語の原型化
                        "kuromoji_part_of_speech", # 不要品詞の除去
                        "ja_stop", #不要単語の除去
                        "kuromoji_number", # 数字の正規化
                        "kuromoji_stemmer" #長音の正規化
                    ]
                },
                "jpn-index": { # インデックス生成用アナライザの定義
                    "type": "custom",
                    "char_filter": [
                        "icu_normalizer", # 文字単位の正規化
                        "kuromoji_iteration_mark" # 繰り返し文字の正規化
                    ],
                    "tokenizer": "kuromoji_w_dic", # 辞書付きkuromoji形態素解析
                    "filter": [
                        "kuromoji_baseform", # 活用語の原型化
                        "kuromoji_part_of_speech", # 不要品詞の除去
                        "ja_stop", #不要単語の除去
                        "kuromoji_number", # 数字の正規化
                        "kuromoji_stemmer" #長音の正規化
                    ]
                }
            }
        }
    }
}

# 日本語用インデックス名の定義
jp_index = 'jp_index'

# 同じ名前のインデックスがすでにあれば削除する
if es.indices.exists(index = jp_index):
    es.indices.delete(index = jp_index)

# インデックス jp_doc の生成
es.indices.create(index = jp_index, body = create_index)

In [ ]:
# リスト 3.3.4 分析結果表示用関数 analyse_jp_test

def analyse_jp_text(text):
    body = {"analyzer": "jpn-search", "text": text}
    ret = es.indices.analyze(index = jp_index, body = body)
    tokens = ret['tokens']
    tokens2 = [token['token'] for token in tokens]
    return tokens2

# analyse_jp_test 関数のテスト
print(analyse_jp_text('関数のテスト'))

In [ ]:
# リスト 3.3.5 icu_normalizerのテスト

print(analyse_jp_text('ｱﾊﾟｰﾄ'))
print(analyse_jp_text('㌀'))

In [ ]:
# リスト 3.3.6 kuromoji_iteration_markのテスト

print(analyse_jp_text('時々'))
print(analyse_jp_text('こゝろ'))
print(analyse_jp_text('学問のすゝめ'))

In [ ]:
# リスト 3.3.7 kuromoji_baseformのテスト

print(analyse_jp_text('昨日、飲みに行った。'))

In [ ]:
# リスト 3.3.8 kuromoji_part_of_speechのテスト

print(analyse_jp_text('この店は寿司がおいしい。'))

In [ ]:
#  リスト 3.3.9 ja_stopのテスト

print(analyse_jp_text('しかし、これでいいのか迷ってしまう。'))

In [ ]:
# リスト 3.3.10 kuromoji_numberのテスト

print(analyse_jp_text('一億二十三'))

In [ ]:
# リスト 3.3.11 kuromoji_stemmerのテスト

print(analyse_jp_text('コンピューターを操作する'))